In [27]:
import numpy as np

In [150]:
# c = [1, 1, 0, 0, 0]
# A = [
#     [-1, 1, 1, 0, 0],
#     [ 1, 0, 0, 1, 0],
#     [ 0, 1, 0, 0, 1]
# ]
# b = [2, 4, 4]

In [151]:
def is_float(element: any) -> bool:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False
    

In [241]:
def read_from_string(string):
    data = string.split('\n')
    allowed_signs = {'<=','>='}
    if data[0] == 'min':
        min_max_multiplicator = -1
    elif data[0] == 'max':
        min_max_multiplicator = 1
        
    else:
        raise AssertionError('Введите направление оптимизации в первой строке файла')
    c = list(map(float, data[1].split()))
    A = []
    b = []
    for row in data[2:]:
        cur_row = []
        cur_sign = 0 
        for val in row.split():
#             try:
#                 val=float(val)
#             except:
#                 pass
#             print(val)
            if is_float(val):
                
                cur_row.append(float(val))
            elif val in allowed_signs and cur_sign == 0:
                if val == '<=':
                    cur_sign = 1
                else:
                    cur_sign = -1
            else:
                raise AssertionError(f'Неправильный формат строки {row}')
        if cur_sign != 0:
            cur_row = np.array(cur_row) * cur_sign
        else:
            raise AssertionError(f'Неправильный формат строки, знак неравенства не найден {row} \n Допустимые форматы знаков неравенств: {allowed_signs}')
        print(cur_row)
        A.append(cur_row[:-1])
        b.append(cur_row[-1])
    A = np.array(A)
    b = np.array(b).reshape(-1,1)
    c = np.array(c).reshape(1, -1) * min_max_multiplicator
    A=A.tolist()
    b=b[:,0].tolist()
    c=c.tolist()[0]
    return A, b, c



In [243]:

# b=[350,240,150,0,0]

[  3.5   1.  350. ]
[  1.   2. 240.]
[  1.   1. 150.]
[-1. -0. -0.]
[-0. -1. -2.]


([[3.5, 1.0], [1.0, 2.0], [1.0, 1.0], [-1.0, -0.0], [-0.0, -1.0]],
 [350.0, 240.0, 150.0, -0.0, -2.0],
 [200.0, 100.0])

In [244]:
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

In [245]:
def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

In [246]:
import math

def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

In [247]:
def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

In [248]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
#         print('col',column)
        solutions.append(solution)
        
    return solutions

In [249]:
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)
    

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)
#         print(tableau)
#     print(tableau)

    return get_solution(tableau)

In [250]:
# class Tableau:
#     def __init__(self, A, b, c):
#         self.mat = np.hstack([np.vstack([np.zeros(1), b]),
#                               np.vstack([c,A])])
#         self.m = self.mat.shape[0]
#         self.n = self.mat.shape[1]
#         self.counter = 0
# Tableau(A,b,c)


In [270]:
A, b, c=read_from_string(
"""max
50 100
2 1 <= 1000
10 20 <= 11000
2 1 <= 10000
1 0 >= 0
0 1 >= 0""")
A, b, c

[   2.    1. 1000.]
[1.0e+01 2.0e+01 1.1e+04]
[2.e+00 1.e+00 1.e+04]
[-1. -0. -0.]
[-0. -1. -0.]


([[2.0, 1.0], [10.0, 20.0], [2.0, 1.0], [-1.0, -0.0], [-0.0, -1.0]],
 [1000.0, 11000.0, 10000.0, -0.0, -0.0],
 [50.0, 100.0])

In [271]:
solution = simplex(c, A, b)
print('solution: ', solution)

solution:  [300.0, 400.0, 0]
